In [1]:
using MMA, Suppressor

In [2]:
function loss(w, grad, X, y)
    obj = 0.0
    grad .= 0
    @inbounds for i in 1:size(X, 2)
        y_pred = w[1]
        for j in 2:length(w)
            y_pred += X[j-1,i] * w[j]
        end
        error = y_pred - y[i]
        for j in 1:length(w)
            if j == 1
                grad[j] += 2*error
            else
                grad[j] += 2*error*X[j-1,i]
            end
        end
        obj += error^2
    end    
    return obj
end

function onenormupper(w, grad, limit)
    grad .= sign.(w)
    return norm(w, 1) - limit
end
function onenormlower(w, grad, limit)
    grad .= -sign.(w)
    return limit - norm(w, 1)
end

onenormlower (generic function with 1 method)

In [3]:
nparams = 10000
npoints = 100

srand(1)
X = rand(nparams, npoints)
l = 5.0
u = 50.0
true_w = full(sprand(nparams, (l+u)/2/nparams))

y = (true_w'*X)'

w0 = rand(nparams);

In [4]:
m = MMAModel(nparams, (w, grad)->loss(w, grad, X, y), xtol = 1e-6, max_iters=200)
# Add box constraints to the variables
box!(m, 0.0, 1.0)

ineq_constraint!(m, (w, grad)->onenormupper(w, grad, u))
ineq_constraint!(m, (w, grad)->onenormlower(w, grad, l))

# Solve the problem
results = @suppress MMA.optimize(m, w0, MMA.MMA02(), Optim.LBFGS(), dual_caps=(0,100))

Results of Optimization Algorithm
 * Algorithm: MMA.MMA02
 * Starting Point: [0.5340869542381113,0.5825223194059725, ...]
 * Minimizer: [0.0,0.0, ...]
 * Minimum: 8.978184e+02
 * Iterations: 39
 * Convergence: true
   * |x - x'| ≤ 1.0e-06: false 
     |x - x'| = 3.92e-03 
   * |f(x) - f(x')| ≤ 1.5e-08 |f(x)|: true
     |f(x) - f(x')| = 6.21e-13 |f(x)|
   * |g(x)| ≤ 1.5e-08: false 
     |g(x)| = 5.77e+02 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 40
 * Gradient Calls: 40

In [5]:
w = results.minimizer
norm(w, 1)

6.1644515502909005

In [6]:
norm(true_w, 1)

11.790602008880104

In [7]:
norm(true_w - w, 1)

17.9302508091891

In [8]:
sum(w .> 0.001)

2624

In [9]:
sum(true_w .> 0.001)

25